In [ ]:
%pip install -q ipywidgets

In [ ]:
%load_ext autoreload
%autoreload now
import tempo_embeddings
import ipywidgets as widgets

#### Connect to Existing Database

In [ ]:
from tempo_embeddings.embeddings.vector_database import ChromaDatabaseManager

db = ChromaDatabaseManager(db_path="testing_db_big", batch_size=24)
db.connect()

#### Choose from Existing Collections in the DB

In [ ]:
existing_colls = db.get_available_collections()
collection_selector = widgets.Dropdown(
    options=existing_colls,
    description='Collection:',
    disabled=False,
)
collection_selector

#### Analyze collections

Number of records in collections:

In [ ]:
collection = db.get_existing_collection(collection_selector.value)
print(collection.count())

In [ ]:
print("Metadatas in collection (value, count) pairs")
stats = db.get_metadata_stats(collection, include_only=["date", "year", "issue"])
for field, val_dict in stats.items():
    print(f"\n----- {field} -----")
    print(sorted(val_dict.items(), key= lambda x: - x[1]))

Plot records by metadata

In [ ]:
## WARNING: The DataFrame only works if the Dataset is of reasonable size (e.g. below 50K records)

corpus = db.get_corpus(collection)
df = corpus.to_dataframe()

print(f"Data Shape = {df.shape}")
df.head()

In [ ]:
field_selector = widgets.Dropdown(
    options=list(df.columns),
    description='Metadata Field:',
    disabled=False,
    value="year"
)
field_selector

In [ ]:
meta_field = field_selector.value
print(f"Distribution by {meta_field}")
print(df[meta_field].value_counts())
df[meta_field].sort_values().hist(xrot=90, figsize=(30,10))